In [1]:
import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def bias_add(V):
    bias = -np.ones( (len(V),1) )
    return np.concatenate( (V,bias), axis=1)
    #return np.hstack( [V,bias] ) # Funciona igual que lo de arriba

def bias_sub(V):
    return V[:,:-1] # Saco la ultima columna

In [ ]:
def activation(X): 
    
    Y = [] # L capas Y_o, Y_1, Y_2, .... Y_l-1
    i = 0
    
    for s in S: # S variable global con los sizes de las capas 
        if i == len(S)-1:
            Y.append( np.zeros((1,s)) )
            i += 1
        else:
            Y.append( np.zeros((1,s+1)) )
            i += 1
        
    Y[0][:] = bias_add(X)
    
    for k in range(1,len(S)-1):
        Y[k][:] = bias_add(Y[k])
        Y[k][:] = np.tanh( np.dot( Y[k-1],W[k] ) ) # W dados (se pueden armar a parte)
        
    Y[-1][:] = np.tanh( np.dot(Y[-2],W[-2]) )
    return Y  

In [ ]:
def correction(Y,Z):
    delta_W = [0] # primer elemento vacio
    
    for i in range(1,S): # S variable global con los sizes de las capas 
        delta_W.append( np.zeros( (S[i-1]+1,S[i]) ) )
        
    # Ver inicializar con size L e insertar en L-1 los valores de abajo
    E = [0]*L # size L 
    D = [0]*L # size L
    d = [0]*L # size L
    
    E[L-1] =  Z - Y[-1]  # En ultima posicion inserto
    d[L-1] = 1 - Y[-1]**2
    D[L-1] = E[-1]*d[-1] 
        
    for k in reversed(np.arange(1,L)):
        delta_w[k] = lr * np.dot( Y[k-1].T, D[k]) # lr = learning rate (variable global)
        E[k] = ( D[k] ) 
        d[k] = ( 1 - Y[k-1]**2 )
        D[k] = ( bias_sub( E[k]*d[k] ) )
    return delta_w